In [25]:
import simpy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Simulation parameters
SIM_DURATION = 30 * 12 * 60  # 30 days, 12 hours per day (8 AM to 8 PM)
CAPACITY = 250
WILLINGNESS_TO_VISIT = 0.80  # 80% willingness to visit

TARGET_LOST_CUSTOMERS_PERCENTAGE = 10  # Target for lost customers due to full capacity
tolerance = 0.5  # Acceptable tolerance for the target percentage
max_iterations = 20  # Maximum number of iterations to adjust the arrival rate

# Arrival rates based on the day of the week and time slots (initial)
ARRIVAL_RATES = {
    'Sunday': [(8, 12, 15), (12, 17, 50), (17, 20, 50)],
    'Monday': [(8, 12, 15), (12, 17, 50), (17, 20, 50)],
    'Tuesday': [(8, 12, 15), (12, 17, 50), (17, 20, 50)],
    'Wednesday': [(8, 12, 15), (12, 17, 50), (17, 20, 50)],
    'Thursday': [(8, 12, 15), (12, 17, 50), (17, 20, 50)],
    'Friday': [(8, 12, 20), (12, 17, 70), (17, 20, 80)],
    'Saturday': [(8, 12, 20), (12, 17, 70), (17, 20, 80)]
}

# Initialize variables
arrival_pattern = np.zeros((30, 12))  # 30 days, 12 hours per day
parking_durations = []  # List to store parking durations
total_revenue = 0  # Total revenue accumulator
lost_customers_full_capacity = 0  # Lost customers due to full capacity
parked_customers = 0  # Parked customers accumulator

def get_arrival_rate(day, hour):
    """Return the arrival rate based on the day and hour."""
    for start, end, rate in ARRIVAL_RATES[day]:
        if start <= hour < end:
            return rate
    return 0

def calculate_charge(parking_duration):
    """Calculate the charge based on parking duration."""
    if parking_duration < 30:
        return 15
    else:
        hours = np.ceil(parking_duration / 60)
        return 25 * hours

def vehicle(env, parking_lot, day_index, hour_index):
    """Simulates vehicle parking behavior, tracks arrivals, parking durations, and calculates revenue."""
    global total_revenue, lost_customers_full_capacity, parked_customers
    
    # Check if the vehicle decides to park based on willingness to visit
    if np.random.random() > WILLINGNESS_TO_VISIT:
        return  # Vehicle leaves without parking
    
    # Generate parking duration (in minutes)
    parking_duration = np.random.exponential(60)  # Exponential distribution with mean 60 minutes
    parking_durations.append(parking_duration)
    
    # Calculate the charge based on parking duration
    charge = calculate_charge(parking_duration)
    total_revenue += charge
    parked_customers += 1

    with parking_lot.request() as request:
        yield request
        
        if len(parking_lot.users) >= CAPACITY:
            lost_customers_full_capacity += 1
            return  # Lost customer due to full capacity

        yield env.timeout(parking_duration)

def arrival_process(env, parking_lot, day, day_index):
    """Simulates the arrival process based on the day and time."""
    while env.now < (day_index + 1) * 12 * 60:
        current_hour = (env.now // 60) % 12 + 8
        rate = get_arrival_rate(day, current_hour)
        hour_index = int(current_hour - 8)
        
        if 0 <= hour_index < 12:
            if rate > 0:
                inter_arrival_time = np.random.exponential(60 / rate)
                yield env.timeout(inter_arrival_time)
                env.process(vehicle(env, parking_lot, day_index, hour_index))
            else:
                yield env.timeout(1)

def calculate_lost_percentage():
    """Calculate the percentage of lost customers due to full capacity."""
    global parked_customers, lost_customers_full_capacity
    total_attempted_arrivals = parked_customers + lost_customers_full_capacity
    return (lost_customers_full_capacity / total_attempted_arrivals) * 100 if total_attempted_arrivals > 0 else 0

def adjust_arrival_rates(day_index):
    """Adjust the arrival rates based on lost customer percentage."""
    global ARRIVAL_RATES
    current_percentage = calculate_lost_percentage()
    if current_percentage < TARGET_LOST_CUSTOMERS_PERCENTAGE:
        adjustment_factor = 1.05
    else:
        adjustment_factor = 0.95
    
    day_name = list(ARRIVAL_RATES.keys())[day_index % 7]
    ARRIVAL_RATES[day_name] = [(start, end, max(1, rate * adjustment_factor)) for start, end, rate in ARRIVAL_RATES[day_name]]

def run_simulation():
    """Runs the simulation for 30 days."""
    global parked_customers, lost_customers_full_capacity, total_revenue
    parked_customers, lost_customers_full_capacity, total_revenue = 0, 0, 0
    env = simpy.Environment()
    parking_lot = simpy.Resource(env, capacity=CAPACITY)

    for day_index in range(30):
        day_name = list(ARRIVAL_RATES.keys())[day_index % 7]
        env.process(arrival_process(env, parking_lot, day_name, day_index))
        env.run(until=(day_index + 1) * 12 * 60)

# Main loop to adjust arrival rates
iteration_count = 0
current_lost_percentage = calculate_lost_percentage()

while abs(current_lost_percentage - TARGET_LOST_CUSTOMERS_PERCENTAGE) > tolerance and iteration_count < max_iterations:
    run_simulation()  # Run the simulation for each iteration
    current_lost_percentage = calculate_lost_percentage()
    adjust_arrival_rates(iteration_count)  # Adjust the rates after each run
    iteration_count += 1

# Print final results
print(f"Final Lost Customers Due to Full Capacity: {lost_customers_full_capacity}")
print(f"Final Percentage of Lost Customers Due to Full Capacity: {current_lost_percentage:.2f}%")
print(f"Total Parked Customers: {parked_customers}")
print(f"Total Revenue: ₹{total_revenue:.2f}")


Final Lost Customers Due to Full Capacity: 0
Final Percentage of Lost Customers Due to Full Capacity: 0.00%
Total Parked Customers: 14058
Total Revenue: ₹496350.00
